In [26]:
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [ ]:
cd '/content/drive/MyDrive/introdiction to DS/FinalProject'

/content/drive/MyDrive/introdiction to DS/FinalProject


In [28]:
data_frame = pd.read_csv("/content/drive/MyDrive/introdiction to DS/FinalProject/data/UsedCarListingCleaning.csv")

In [ ]:
data_frame.dtypes

Unnamed: 0          int64
name               object
make               object
model              object
price               int64
year                int64
exterior_color     object
interior_color     object
mileage             int64
num_seats           int64
num_doors         float64
origin             object
style              object
transmission       object
drivetrain         object
fuel               object
liter             float64
dtype: object

In [ ]:
del data_frame['Unnamed: 0']
del data_frame['name']
data_frame['year'] = data_frame['year'].astype(object)
data_frame

,make,model,price,year,exterior_color,interior_color,mileage,num_seats,num_doors,origin,style,transmission,drivetrain,fuel,liter
0,Ford,Everest,950,2022,Đen,Đen,30000,7,5.0,Nhập khẩu,SUV,Số tự động,RFD - Dẫn động cầu sau,Dầu,2.00
1,Lexus,RX,1990,2017,Trắng,Đen,50000,5,5.0,Nhập khẩu,SUV,Số tự động,AWD - 4 bánh toàn thời gian,Xăng,2.00
2,Mercedes Benz,S class,2790,2020,Trắng,Nâu,30000,5,4.0,Lắp ráp trong nước,Sedan,Số tự động,RFD - Dẫn động cầu sau,Xăng,3.00
3,Toyota,Vios,448,2019,Đen,Kem,46000,5,4.0,Lắp ráp trong nước,Sedan,Số tự động,FWD - Dẫn động cầu trước,Xăng,1.50
4,Volvo,S60,1279,2021,Trắng,Đen,19000,5,4.0,Nhập khẩu,Sedan,Số tự động,AWD - 4 bánh toàn thời gian,Xăng,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24636,Ford,Ranger,500,2015,Bạc,Đen,0,5,4.0,Nhập khẩu,Bán tải / Pickup,Số tay,RFD - Dẫn động cầu sau,Dầu,2.20
24637,Chevrolet,Lacetti,195,2011,Vàng,Ghi,0,5,4.0,Lắp ráp trong nước,Sedan,Số tay,FWD - Dẫn động cầu trước,Xăng,1.60
24638,Kia,Morning,220,2013,Trắng,Đen,0,5,5.0,Lắp ráp trong nước,Hatchback,Số tay,FWD - Dẫn động cầu trước,Xăng,1.25
24639,Kia,Morning,365,2016,Trắng,Xám,0,5,5.0,Lắp ráp trong nước,Hatchback,Số tự động,FWD - Dẫn động cầu trước,Xăng,1.25


In [ ]:
unique_values_array = {}

for column in data_frame.select_dtypes(include=['object']).columns:
  print(column)
  unique_values = data_frame[column].unique()
  unique_values_array[column] = unique_values.tolist()
  if column.__eq__('year'):
    unique_values_array[column] = [str(value) for value in unique_values_array]
  print(unique_values)

make
['Ford' 'Lexus' 'Mercedes Benz' 'Toyota' 'Volvo' 'VinFast' 'LandRover'
 'BMW' 'Kia' 'Hyundai' 'Porsche' 'Mazda' 'Suzuki' 'Daewoo' 'Peugeot'
 'Mitsubishi' 'Audi' 'Nissan' 'Mini' 'Honda' 'Bentley' 'Chevrolet'
 'Subaru' 'Isuzu' 'Dongben' 'MG' 'Daihatsu' 'Thaco' 'Acura' 'Volkswagen'
 'Maserati' 'Dodge' 'Jeep' 'Rolls Royce' 'Cadillac' 'Infiniti'
 'Lamborghini' 'Renault' 'Citroen' 'Hummer' 'Lincoln' 'Jaguar' 'Zotye'
 'Ssangyong' 'Baic' 'SYM' 'Ferrari' 'Hino' 'Scion' 'Fiat' 'McLaren' 'RAM'
 'Dongfeng' 'Maybach' 'Tobe' 'Hongqi' 'Smart' 'Mekong' 'JRD' 'Samsung'
 'Vinaxuki' 'Haima' 'Lifan' 'Genesis' 'Luxgen' 'Aston Martin' 'Chrysler'
 'UAZ' 'Chery' 'Changan' 'Brilliance']
model
['Everest' 'RX' 'S class' 'Vios' 'S60' 'VF e34' 'Range Rover' 'Ranger'
 'X5' 'Sedona' 'SantaFe' 'GLC' '718' 'Cayenne' '2' 'Panamera' 'Kona'
 'Swift' 'Fadil' 'CX5' 'C class' 'Lacetti' 'Fortuner' 'Tucson' '3008'
 'Morning' 'Mustang' 'Pajero Sport' 'A6' 'V class' 'E class' '3' 'Cerato'
 'Almera' 'K5' 'Sonet' 'Cooper' 'B

# **Test**

In [ ]:
data = {
    'make': ["Chevrolet", "Mercedes Benz", "Mercedes Benz"],
    'model': ["Spark", "GLC", "E class"],
    'year': [2009, 2020, 2025],
    'exterior_color': ["Bạc", "Trắng", "Trắng"],
    'interior_color': ["Ghi", "Nâu", "Đen"],
    'mileage': [268758, 46000, 40000],
    'num_seats': [5, 5, 5],
    'num_doors': [5, 5, 4],
    'origin': ["Nhập khẩu", "Nhập khẩu", "Lắp ráp trong nước"],
    'style': ["SUV", "SUV", "Sedan"],
    'transmission': ["Số tự động", "Số tự động", "Số tự động"],
    'drivetrain': ["FWD - Dẫn động cầu trước", "AWD - 4 bánh toàn thời gian", "RFD - Dẫn động cầu sau"],
    'fuel': ['Xăng', 'Xăng', 'Xăng'],
    'liter': [0.8, 2, 2]
}

df = pd.DataFrame(data)
df['year'] = df['year'].astype(object)

df

,make,model,year,exterior_color,interior_color,mileage,num_seats,num_doors,origin,style,transmission,drivetrain,fuel,liter
0,Chevrolet,Spark,2009,Bạc,Ghi,268758,5,5,Nhập khẩu,SUV,Số tự động,FWD - Dẫn động cầu trước,Xăng,0.8
1,Mercedes Benz,GLC,2020,Trắng,Nâu,46000,5,5,Nhập khẩu,SUV,Số tự động,AWD - 4 bánh toàn thời gian,Xăng,2.0
2,Mercedes Benz,E class,2025,Trắng,Đen,40000,5,4,Lắp ráp trong nước,Sedan,Số tự động,RFD - Dẫn động cầu sau,Xăng,2.0


In [ ]:
loaded_label_encoders = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/label_encoders.pkl')

for column in df.select_dtypes(include=['object']).columns:
    loaded_label_encoder = loaded_label_encoders[column]
    df[column] = loaded_label_encoder.transform(df[column])

In [ ]:
loaded_model_xgb = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/xgboost_model.joblib')
loaded_label_encoders = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/label_encoders.pkl')
df = label_encoder(loaded_label_encoders, df)
predictions = loaded_model_xgb.predict(df)
predictions

In [ ]:
loaded_model_rf = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/random_forest_model.joblib')
predictions = loaded_model_rf.predict(df)
predictions

array([1721.85716667])

In [ ]:
predict("XGBoost", "Ford", "Everest", 2018, "Đỏ", "Đen", 79000, 7, 5, "Nhập khẩu", "SUV", "Số tự động", "RFD - Dẫn động cầu sau", "Dầu", 2)

'836.89606'

# **User Interface**

In [ ]:
!pip install -q gradio

In [ ]:
rf_model = "/content/drive/MyDrive/introdiction to DS/FinalProject/models/random_forest_model.joblib"
xgboost_model = "/content/drive/MyDrive/introdiction to DS/FinalProject/models/xgboost_model.joblib"

In [ ]:
def create_data_frame(make, model, year, external_color, interior_color, mileage, num_seats,
            num_doors, origin, style, transmission, drivetrain, fuel, liter):
    data = {
      'make': [make],
      'model': [model],
      'year': [year],
      'exterior_color': [external_color],
      'interior_color': [interior_color],
      'mileage': [mileage],
      'num_seats': [num_seats],
      'num_doors': [num_doors],
      'origin': [origin],
      'style': [style],
      'transmission': [transmission],
      'drivetrain': [drivetrain],
      'fuel': [fuel],
      'liter': [liter]
    }

    df1 = pd.DataFrame(data)
    df1['year'] = df1['year'].astype(object)
    df1
    return df1


In [ ]:
def label_encoder(loaded_label_encoders, df):
  for column in df.select_dtypes(include=['object']).columns:
      loaded_label_encoder = loaded_label_encoders[column]
      try:
          df[column] = loaded_label_encoder.transform(df[column])
      except Exception as e:
          return column
          break
  return df

In [ ]:
def predict(models, make, model, year, external_color, interior_color, mileage, num_seats,
            num_doors, origin, style, transmission, drivetrain, fuel, liter):
    model_path = ""
    if models.__eq__('Random forest'):
        model_path = rf_model
    else:
        model_path = xgboost_model
    loaded_model = joblib.load(model_path)
    df = create_data_frame(make, model, year, external_color, interior_color, mileage, num_seats,
            num_doors, origin, style, transmission, drivetrain, fuel, liter)
    loaded_label_encoders = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/label_encoders.pkl')
    df = label_encoder(loaded_label_encoders, df)
    if isinstance(df, str):
        return "Value " + df + " is not true"
    if num_seats <= 0:
        return "Value number of seats is not true"
    if num_doors <= 0:
        return "Value number of doors is not true"
    if liter <= 0:
        return "Value number of fuel liter is not true"
    predictions = loaded_model.predict(df)
    return str(predictions[0])

In [ ]:
def check_columns(df):
    required_columns = ['make', 'model', 'year', 'exterior_color', 'interior_color',
                         'mileage', 'num_seats', 'num_doors', 'origin', 'style',
                         'transmission', 'drivetrain', 'fuel', 'liter']

    missing_columns = [col for col in required_columns if col not in df.columns]

    if not missing_columns:
        return True
    else:
        return False


In [ ]:
def predictFile(models, file_path):
    model_path = ""
    result = ""
    if models.__eq__('Random forest'):
        model_path = rf_model
    else:
        model_path = xgboost_model
    loaded_model = joblib.load(model_path)
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
        if 'price' in df.columns:
            df = df.drop(columns=['price'])
        if check_columns(df):
            loaded_label_encoders = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/label_encoders.pkl')
            df = label_encoder(loaded_label_encoders, df)
            predictions = loaded_model.predict(df)
            list_predictions = list(predictions)
            for x in list_predictions:
                result = result + str(round(x, 2)) + "    "
            return result
        else:
            return 0
    else:
        return 0


In [ ]:
import gradio as gr

demo = gr.Blocks()
models = ["Random forest", "XGBoost"]

with demo:
    gr.Markdown("Prediction car demo.")
    with gr.Tab("Input information"):
        models_input = gr.Dropdown(models, label="Algorithm")
        with gr.Row():
            make_input = gr.Dropdown(unique_values_array['make'], label="Make")
            model_input = gr.Dropdown(unique_values_array['model'], label="Model")
        with gr.Row():
            year_input = gr.Number(label="Year (must be from 1989 to 2023)", value=2023)
            mileage_input = gr.Number(label="Mileage")
        with gr.Row():
            external_color_input = gr.Dropdown(unique_values_array['exterior_color'], label="Exterior Color")
            interior_color_input = gr.Dropdown(unique_values_array['interior_color'], label="Interior Color")
        with gr.Row():
            num_seats_input = gr.Number(label="Num Seats")
            num_doors_input = gr.Number(label="Num Doors")
        with gr.Row():
            origin_input = gr.Dropdown(unique_values_array['origin'], label="Origin")
            style_input = gr.Dropdown(unique_values_array['style'], label="Style")
        with gr.Row():
            transmission_input = gr.Dropdown(unique_values_array['transmission'], label="Transmission")
            drivetrain_input = gr.Dropdown(unique_values_array['drivetrain'], label="Drive Train")
        with gr.Row():
            fuel_input = gr.Dropdown(unique_values_array['fuel'], label="Fuel")
            liter_input = gr.Number(label="Liter", step=0.1)
        predict_button = gr.Button(value = "Predict")
        price_output = gr.Textbox(label="Price")

    with gr.Tab("Import file"):
        file = gr.File(label="Upload a File")
        models_input = gr.Dropdown(models, label="Algorithm")
        predict_button_2 = gr.Button(value = "Predict")
        price_output_2 = gr.Textbox(label="Price")

    with gr.Accordion("Open for More!"):
        gr.Markdown("Look at me...")

    predict_button.click(predict, [models_input, make_input, model_input, year_input, external_color_input, interior_color_input,
                           mileage_input, num_seats_input, num_doors_input, origin_input, style_input,
                           transmission_input, drivetrain_input, fuel_input, liter_input], price_output)
    predict_button_2.click(predictFile, [models_input, file], price_output_2)

if __name__ == "__main__":
    demo.launch()


# **Code deploy**

In [ ]:
import pandas as pd
import joblib
import xgboost as xgb
import gradio as gr

rf_model = "/content/drive/MyDrive/introdiction to DS/FinalProject/models/random_forest_model.joblib"
xgboost_model = "/content/drive/MyDrive/introdiction to DS/FinalProject/models/xgboost_model.json"

data_frame = pd.read_csv("/content/drive/MyDrive/introdiction to DS/FinalProject/data/UsedCarListingCleaning.csv")
del data_frame['Unnamed: 0']
del data_frame['name']
data_frame['year'] = data_frame['year'].astype(object)

unique_values_array = {}

for column in data_frame.select_dtypes(include=['object']).columns:
  unique_values = data_frame[column].unique()
  unique_values_array[column] = unique_values.tolist()
  if column.__eq__('year'):
    unique_values_array[column] = [str(value) for value in unique_values_array]

def create_data_frame(make, model, year, external_color, interior_color, mileage, num_seats,
            num_doors, origin, style, transmission, drivetrain, fuel, liter):
    data = {
      'make': [make],
      'model': [model],
      'year': [year],
      'exterior_color': [external_color],
      'interior_color': [interior_color],
      'mileage': [mileage],
      'num_seats': [num_seats],
      'num_doors': [num_doors],
      'origin': [origin],
      'style': [style],
      'transmission': [transmission],
      'drivetrain': [drivetrain],
      'fuel': [fuel],
      'liter': [liter]
    }

    df1 = pd.DataFrame(data)
    df1['year'] = df1['year'].astype(object)
    return df1

def label_encoder(loaded_label_encoders, df):
  for column in df.select_dtypes(include=['object']).columns:
      loaded_label_encoder = loaded_label_encoders[column]
      try:
          df[column] = loaded_label_encoder.transform(df[column])
      except Exception as e:
          return column
  return df

def predict_one(models, make, model, year, external_color, interior_color, mileage, num_seats,
            num_doors, origin, style, transmission, drivetrain, fuel, liter):
    model_path = ""
    loaded_model = ""
    if models.__eq__('Random forest'):
        model_path = rf_model
        loaded_model = joblib.load(model_path)
    else:
        model_path = xgboost_model
        loaded_model = xgb.XGBRegressor()
        loaded_model.load_model(model_path)

    df = create_data_frame(make, model, year, external_color, interior_color, mileage, num_seats,
            num_doors, origin, style, transmission, drivetrain, fuel, liter)
    loaded_label_encoders = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/label_encoders.pkl')
    df = label_encoder(loaded_label_encoders, df)
    if isinstance(df, str):
        return "Value " + df + " is not true"
    if num_seats <= 0:
        return "Value number of seats is not true"
    if num_doors <= 0:
        return "Value number of doors is not true"
    if liter <= 0:
        return "Value number of fuel liter is not true"
    predictions = loaded_model.predict(df)
    return str(predictions[0])

def check_columns(df):
    required_columns = ['make', 'model', 'year', 'exterior_color', 'interior_color',
                         'mileage', 'num_seats', 'num_doors', 'origin', 'style',
                         'transmission', 'drivetrain', 'fuel', 'liter']

    missing_columns = [col for col in required_columns if col not in df.columns]

    if not missing_columns:
        return True
    else:
        return False

def predictFile(models, file_path):
    model_path = ""
    result = ""
    loaded_model = ""
    if models.__eq__('Random forest'):
        model_path = rf_model
        loaded_model = joblib.load(model_path)
    else:
        model_path = xgboost_model
        loaded_model = xgb.XGBRegressor()
        loaded_model.load_model(model_path)

    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
        if 'price' in df.columns:
            df = df.drop(columns=['price'])
        if check_columns(df):
            loaded_label_encoders = joblib.load('/content/drive/MyDrive/introdiction to DS/FinalProject/models/label_encoders.pkl')
            df = label_encoder(loaded_label_encoders, df)
            predictions = loaded_model.predict(df)
            list_predictions = list(predictions)
            for x in list_predictions:
                result = result + str(round(x, 2)) + "    "
            return result
        else:
            return 0
    else:
        return 0

demo = gr.Blocks()
models = ["Random forest", "XGBoost"]

with demo:
    gr.Markdown("Prediction car demo.")
    with gr.Tab("Input information"):
        models_input = gr.Dropdown(models, label="Algorithm")
        with gr.Row():
            make_input = gr.Dropdown(unique_values_array['make'], label="Make")
            model_input = gr.Dropdown(unique_values_array['model'], label="Model")
        with gr.Row():
            year_input = gr.Number(label="Year (must be from 1989 to 2023)", value=2023)
            mileage_input = gr.Number(label="Mileage")
        with gr.Row():
            external_color_input = gr.Dropdown(unique_values_array['exterior_color'], label="Exterior Color")
            interior_color_input = gr.Dropdown(unique_values_array['interior_color'], label="Interior Color")
        with gr.Row():
            num_seats_input = gr.Number(label="Num Seats")
            num_doors_input = gr.Number(label="Num Doors")
        with gr.Row():
            origin_input = gr.Dropdown(unique_values_array['origin'], label="Origin")
            style_input = gr.Dropdown(unique_values_array['style'], label="Style")
        with gr.Row():
            transmission_input = gr.Dropdown(unique_values_array['transmission'], label="Transmission")
            drivetrain_input = gr.Dropdown(unique_values_array['drivetrain'], label="Drive Train")
        with gr.Row():
            fuel_input = gr.Dropdown(unique_values_array['fuel'], label="Fuel")
            liter_input = gr.Number(label="Liter", step=0.1)
        predict_button = gr.Button(value = "Predict")
        price_output = gr.Textbox(label="Price")

    with gr.Tab("Import file"):
        file = gr.File(label="Upload a File")
        models_input_2 = gr.Dropdown(models, label="Algorithm")
        predict_button_2 = gr.Button(value = "Predict")
        price_output_2 = gr.Textbox(label="Price")

    with gr.Accordion("Open for More!"):
        gr.Markdown("Look at me...")

    predict_button.click(predict_one, [models_input, make_input, model_input, year_input, external_color_input, interior_color_input,
                           mileage_input, num_seats_input, num_doors_input, origin_input, style_input,
                           transmission_input, drivetrain_input, fuel_input, liter_input], price_output)
    predict_button_2.click(predictFile, [models_input_2, file], price_output_2)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7bb34d06dfa807a144.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
predict("xgb", "Ford", "Everest", 2018, "Đỏ", "Đen", 79000, 7, 5, "Nhập khẩu", "SUV", "Số tự động", "RFD - Dẫn động cầu sau", "Dầu", 2)

'836.89606'